# Wikipedia Retrieval Augmented Generation

In [1]:
%pip install openai
%pip install python-dotenv
%pip install wikipedia-api

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
import wikipediaapi
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
wiki = wikipediaapi.Wikipedia('Test (email@example.com)', 'en')

def getWikipediaPage(name):
    """Get the Wikipedia page given its name"""
    return wiki.page(name).text

In [4]:
print(getWikipediaPage("Hurricane_Otis"))

Hurricane Otis was a compact yet devastating tropical cyclone which made landfall in October 2023 near Acapulco as a Category 5 hurricane. Otis was the first Pacific hurricane to make landfall at Category 5 intensity and surpassed Hurricane Patricia as the strongest landfalling Pacific hurricane on record. The fifteenth tropical storm, tenth hurricane, eighth major hurricane, and second Category 5 hurricane of the 2023 Pacific hurricane season, Otis originated from a disturbance several hundred miles south of the Gulf of Tehuantepec. Initially forecast to only be a weak tropical storm at peak intensity, Otis instead underwent explosive intensification to reach peak winds of 165 mph (270 km/h) and made landfall at peak intensity. Once inland, the hurricane rapidly weakened, before dissipating the following day. 
Making landfall just west of Acapulco, Otis' powerful winds damaged many of the buildings in the city. Landslides occurred, and flooding resulted from continuous heavy rain. Com

In [5]:
functions = [
    {
        "name": "getWikipediaPage",
        "description": "Get the Wikipedia page given its name",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Page name",
                },
                "page": {"type": "string"},
            },
            "required": ["location"],
        },
    }
]

In [6]:
messages = [
    {
        "role": "user",
        "content": "What is teh category of Hurricane Otis?"
    }
]

In [7]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [8]:
print(response)

{
  "id": "chatcmpl-8ElfOMvgaW9E661OMNMEY7VQUROGX",
  "object": "chat.completion",
  "created": 1698530514,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "getWikipediaPage",
          "arguments": "{\n  \"name\": \"Hurricane_Otis\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 66,
    "completion_tokens": 21,
    "total_tokens": 87
  }
}


In [9]:
response_message = response["choices"][0]["message"]

In [10]:
json.loads(response_message["function_call"]["arguments"])

{'name': 'Hurricane_Otis'}

In [11]:
args = json.loads(response_message["function_call"]["arguments"])

In [12]:
wikipediaText = getWikipediaPage(args['name'])
print(wikipediaText)

Hurricane Otis was a compact yet devastating tropical cyclone which made landfall in October 2023 near Acapulco as a Category 5 hurricane. Otis was the first Pacific hurricane to make landfall at Category 5 intensity and surpassed Hurricane Patricia as the strongest landfalling Pacific hurricane on record. The fifteenth tropical storm, tenth hurricane, eighth major hurricane, and second Category 5 hurricane of the 2023 Pacific hurricane season, Otis originated from a disturbance several hundred miles south of the Gulf of Tehuantepec. Initially forecast to only be a weak tropical storm at peak intensity, Otis instead underwent explosive intensification to reach peak winds of 165 mph (270 km/h) and made landfall at peak intensity. Once inland, the hurricane rapidly weakened, before dissipating the following day. 
Making landfall just west of Acapulco, Otis' powerful winds damaged many of the buildings in the city. Landslides occurred, and flooding resulted from continuous heavy rain. Com

In [13]:
messages.append(
        {
            "role": "function",
            "name": "getWikipediaPage",
            "content": wikipediaText,
        }
)

In [14]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages
)
print(response['choices'][0]['message']['content'])

Hurricane Otis was a Category 5 hurricane.
